In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [2]:
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()  # This loads the variables from the .env file
hf_token = os.getenv("HF_TOKEN")

In [1]:
import os
import shutil
import logging
import sagemaker
#import globals as g
import requests as req
from typing import Dict
from pathlib import Path
#from utils import get_bucket_name
from sagemaker.s3 import S3Uploader
from sagemaker import get_execution_role
from huggingface_hub import snapshot_download
from sagemaker.huggingface.model import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::084375588776:role/service-role/AmazonSageMaker-ExecutionRole-20250210T195773
sagemaker bucket: sagemaker-us-west-2-084375588776
sagemaker session region: us-west-2


In [5]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.1.0",
  session=sess,
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04


#### Time to download the model:

In [8]:
from huggingface_hub import snapshot_download
HF_MODEL_ID="DAMO-NLP-SG/VideoLLaMA3-7B"

model_dir = Path(HF_MODEL_ID.split("/")[-1])
model_dir.mkdir(exist_ok=True)
# Download model from Hugging Face into model_dir
snapshot_download(HF_MODEL_ID, local_dir=str(model_dir), local_dir_use_symlinks=False)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.98k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

...

processor_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.50k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

'/home/sagemaker-user/PerceiveAI/SageMaker/VideoLLaMA3-7B'

In [22]:
%cd ..
!ls

/home/sagemaker-user/PerceiveAI/SageMaker
0_deploy_llava.ipynb  VideoLLaMA3-7B  deploy_llava.ipynb  deployment.ipynb


In [25]:
%%time

!tar cvzf model.tar.gz -C VideoLLaMA3-7B/ .

./
./.cache/
./.cache/huggingface/
./.cache/huggingface/.gitignore
./.cache/huggingface/download/
./.cache/huggingface/download/.gitattributes.lock
./.cache/huggingface/download/.gitattributes.metadata
./.cache/huggingface/download/README.md.lock
./.cache/huggingface/download/README.md.metadata
./.cache/huggingface/download/added_tokens.json.lock
./.cache/huggingface/download/added_tokens.json.metadata
./.cache/huggingface/download/chat_template.json.lock
./.cache/huggingface/download/chat_template.json.metadata
./.cache/huggingface/download/config.json.lock
./.cache/huggingface/download/config.json.metadata
./.cache/huggingface/download/configuration_videollama3.py.lock
./.cache/huggingface/download/configuration_videollama3.py.metadata
./.cache/huggingface/download/configuration_videollama3_encoder.py.lock
./.cache/huggingface/download/configuration_videollama3_encoder.py.metadata
./.cache/huggingface/download/generation_config.json.lock
./.cache/huggingface/download/generation_confi

In [27]:
!rm model.tar.gz

rm: cannot remove 'model.tar.gz': No such file or directory


In [ ]:

from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_model_uri,      # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.28.1",  # transformers version used
   pytorch_version="2.0.0",       # pytorch version used
   py_version='py310',            # python version used
   model_server_workers=1
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
    # container_startup_health_check_timeout=600, # increase timeout for large models
    # model_data_download_timeout=600, # increase timeout for large models
)